In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import haversine as hs

In [2]:
edges =[['KA', 'DK', 0.1], ['LA', 'NI', 0.1], ['ME', 'ST', 0.2], ['177', 'BV', 0.3], ['216', 'BV', 0.4],
        ['BPG', '216', 0.6], ['WVB', 'WVG', 0.2], ['WVH', 'WVG', 0.15], ['WVF', 'WVG', 0.15],
        ['WI', 'WVG', 0.15], ['WVF', 'WVB', 0.1], ['AF', 'WVF', 0.1], ['WVE', 'WPG', 0.05], ['WPG', 'BK', 0.05],
        ['AF', 'BK', 0.3], ['WI', 'AF', 0.2], ['WI', 'CG', 0.4], ['HO', 'ME', 0.4], ['LA', 'HO', 0.1],
        ['KA', 'LA', 0.1], ['CG', 'KA', 0.1], ['ST', 'CG', 0.1], ['AS', 'RY', 0.6], ['AS', 'RPG', 0.4],
        ['CPG', 'RB', 0.4], ['ISEC', 'SL', 0.4], ['DG', 'EL', 0.2], ['EV', 'CN', 0.2], ['HA', 'RI', 0.2],
        ['MU', 'HS', 0.2], ['HT', 'CN', 0.1], ['MU', 'RB', 0.2], ['CSC', 'EL', 0.1],  ['HA', 'SL', 0.4],
        ['SN', 'CH', 0.2], ['RB', 'HT', 0.3], ['CSC', 'RB', 0.3], ['SL', 'CSC', 0.3], ['SN', 'SL', 0.3],
        ['SN', 'DA', 0.1], ['MA', 'GPG', 0.5], ['CN', 'GPG', 0.2], ['MU', 'CN', 0.2], ['EL', 'MU', 0.2],
        ['HA', 'EL', 0.2], ['CH', 'HA', 0.2], ['FR', 'CH', 0.2], ['106-122', '337', 0.1], ['LH', 'FC', 0.1],
        ['106-122', 'LH', 0.4], ['140', 'SW', 0.4], ['CA', 'SW', 0.4], ['KH', 'MH', 0.1], ['CU', 'KH', 0.1],
        ['CA', 'CU', 0.1], ['KDY', 'AC', 0.1], ['SM', 'KDY', 0.2], ['LF', 'SM', 0.3], ['140', 'LF', 0.4],
        ['SE', '142-148', 0.4], ['SW', 'SE', 0.2], ['WH', 'SW', 0.4], ['407', 'WH', 0.2], ['WVB', 'WVA', 0.3],
        ['WVC', 'WVB', 0.3], ['WVC', 'WVA', 0.3], ['WVE', 'WVC', 0.3], ['RY', 'WVE', 0.3], ['DK', 'FR', 0.2],
        ['DK', 'CB', 0.4], ['NI', 'LT', 0.1], ['SH', 'EC', 0.5], ['DK', 'KN', 0.2], ['NI', 'DK', 0.2],
        ['SH', 'NI', 0.2], ['FR', 'CB', 0.3], ['LT', 'FR', 0.1], ['DA', 'LT', 0.1],['EC', 'DA', 0.2],
        ['EC', 'AS', 0.2], ['768', 'CV', 0.1], ['DCA', 'DCB', 0.1], ['CP', 'DCA', 0.1], ['LV', 'CP', 0.1],
        ['768', 'LV', 0.1], ['768', '780', 0.2], ['RP', '780', 0.5], ['INVO', 'RP', 0.2], ['INV', 'INVO', 0.1],
        ['SB', 'CP', 0.1], ['CPG', 'LV', 0.1], ['ISEC', '768', 0.1], ['RPG', 'RP', 0.1], ['CPG', 'SB', 0.1],
        ['ISEC', 'CPG', 0.2], ['RPG', 'ISEC', 0.6], ['HS', '319', 0.4], ['CB', 'MC', 0.4], ['RI', 'SP', 0.4],
        ['EV', '236', 20], ['HS', 'EV', 0.2], ['HS', 'DG', 0.1], ['RI', 'DG', 0.3], ['RI', 'CB', 0.3],
        ['KN', 'CB', 0.3], ['KN', 'CG', 0.1], ['ST', 'CG', 0.1], ['ST', 'WVH', 0.3], ['450', 'WVH', 0.1],
        ['450', 'BU', 0.05], ['RU', 'BU', 0.05], ['BPG', '177', 0.6], ['BPG', '271', 0.6],['319', '271', 0.4],
        ['337', '319', 0.3], ['337', 'SP', 0.2], ['MC', 'SP', 0.2],['407', 'WH', 0.1], ['WH', 'MC', 0.2]]



In [3]:
# Create list for buildings on the same street
huntington_ave_side_1 = ["RU", "BU", "450","WVH", "ST", "CG", "KN", "CB","RI","DG","HS","EV","236"]
huntington_ave_side_2 = ["407","WH","MC","SP","337","319","271","BPG","177"]
forsyth_st_side_1 = ["EC","AS","DA","LT","FR","CB"]
forsyth_st_side_2 = ["SH","NI","DK","KN"]
ruggle_st = ["RY","WVE","WVC","WVA"]
comlumbus_st_side_1 = ["INV","INVO","RP","780","768","LV","CP","DCA","DCB"]
comlumbus_st_side_2 = ["RPG","ISEC","SB","CPG"]
hemenway_st_back = ["CA","CU","KH","MH"]
hemenway_st_side_1 = ["140","LF","SM","KDY","AC"]
hemenway_st_side_2 = ["407","WH","SW","SE","142-148"]

In [4]:
# Find index of building based on the location of building name in the list
def find_location(stops,stop):
    for i in range(len(stops)):
        if stops[i]== stop:
            return i

In [5]:
def create_edge_for_sites_on_same_street(street_name,edges):
    # create direct edges for all the buildings that are on the same street
    n = len(street_name)
    distance = [[float("Inf") for _ in range (n)] for _ in range(n)]
    for i in range(n):
        distance[i][i] = 0
    for u, v, w in edges:
        if (u in street_name) and (v in street_name):
            u_index = find_location(street_name,u)
            v_index = find_location(street_name,v)
            distance[u_index][v_index] = w
            distance[v_index][u_index] = w
    # Apply Floyd Warshall algorithm to find the sum of shortest path between any two sites
    for k in range(n):
        for i in range(n):
            for j in range(i,n):
                distance[i][j] = min(distance[i][j],distance[i][k] + distance[k][j]) 
                distance[j][i] = min(distance[j][i],distance[j][k] + distance[k][i]) 
    street_edges = []
    for i in range(n):
        for j in range(i+1,n):
            street_edges.append([street_name[i],street_name[j],distance[i][j]])
    return street_edges

In [6]:
def add_edges_for_street(street_name, edges):
    # build a complete graph for all building on the same street
    # Add these edges to the full adjacency list of graph G
    street_edges = create_edge_for_sites_on_same_street(street_name,edges)
    for i in range(len(street_edges)):
        edges.append(street_edges[i])
    return 

In [7]:
add_edges_for_street(huntington_ave_side_1, edges)
add_edges_for_street(huntington_ave_side_2, edges)
add_edges_for_street(forsyth_st_side_1, edges)
add_edges_for_street(forsyth_st_side_2, edges)
add_edges_for_street(comlumbus_st_side_1, edges)
add_edges_for_street(comlumbus_st_side_2, edges)
add_edges_for_street(hemenway_st_back, edges)
add_edges_for_street(hemenway_st_side_1, edges)
add_edges_for_street(hemenway_st_side_2, edges)
add_edges_for_street(ruggle_st, edges)

In [8]:
graph = defaultdict(list)

In [9]:
# Convert a list of edges to a adjancy list representation of graph
for u,v,e in edges: 
    graph[u].append([v,e])
    graph[v].append([u,e])

In [10]:
def distance_f_src_t_dst_w_t_edges(graph, edges, src, dst, t):
    # Apply bellman ford algorithm to find out if dst can by reached from src using at most t edges
    stops = sorted(list(graph.keys()))
    n = len(stops)
    src_index = find_location(stops,src)
    
    distance = [[float("Inf") for _ in range(n)] for _ in range(t+2)]
    
    # set distance to src to 0 and distance from src to other nodes to infinity
    for i in range(t+2):
        distance[i][src_index] = 0
        
    # Find the distance from src to any node with at most i edges
    #,which is same as number of turns people would like to make while walking.  
    for i in range(t+1):
        for u,v,w in edges:
            u_index = find_location(stops,u)
            v_index = find_location(stops,v)
            if distance[i][u_index] != float("Inf"):
                distance[i+1][v_index] = min(distance[i+1][v_index],distance[i][u_index] + w) 
            if distance[i][v_index] != float("Inf"):
                distance[i+1][u_index] = min(distance[i+1][u_index],distance[i][v_index] + w) 
    
    # find the index of dst
    dst_index = find_location(stops,dst)
    
    # if distance to destination with at most t edges is not infinity, then return distance
    # otherwise return infinity
    
    if distance[t+1][dst_index] != float("Inf"):
        return distance[t+1][dst_index]
    else: 
        return float("Inf")

In [11]:
parking_spots ={
    'CPG': (42.33818,-71.086397),
    'GPG': ( 42.340394,-71.085729 ),
    'RPG': (42.336297,-71.088421 ),
    'BPG': (42.343277,-71.085169 ),
    'WPG': (42.336748,-71.091634 ),
    'FC' : (42.342056, -71.08786 ),
    'MA' : (42.341267,-71.084451 ),
    '142-148' : (42.342071, -71.090227 ),
    '140':  ( 42.340789,-71.091759 )
}

In [12]:
def find_driving_distance(parking_spots,current_location):
    # find driving distance from current location to all parking spot
    # return a dictionary of driving distance
    # key = name of parking spot
    # value = Euclidean distance from parking spot to current location in miles based on lat and long
    driving_distance = defaultdict(int)
    keys = parking_spots.keys()
    for k in keys:
        km = hs.haversine(driving_distance[k],current_location)
        mile = km*0.621371
        driving_distance[k]= mile
        # returns distance in km
    return driving_distance

In [13]:
def find_walking_distance(graph, edges, dst, parking_spots,number_of_turns):
    walking_distance = defaultdict(int)
    keys = parking_spots.keys()
    for k in keys:
        d = distance_f_src_t_dst_w_t_edges(graph, edges, k, dst, number_of_turns)
        walking_distance[k] = d
    return walking_distance

In [14]:
def find_most_convenient_parking_location(graph, edges, dst, parking_spots,number_of_turns):
    # return the parkinglocation with the minimal walking distance
    walking_distance = defaultdict(int)
    keys = parking_spots.keys()
    best_location = "None"
    min_distance = float("Inf")
    
    for k in keys:
        d = distance_f_src_t_dst_w_t_edges(graph, edges, k, dst, number_of_turns)
        walking_distance[k] = d
        # if better parking location is found
        # update min_distance and best_location
        if d < min_distance:
            min_distance = d
            best_location = k
    return best_location

In [15]:
find_most_convenient_parking_location(graph, edges, "SL", parking_spots,4)

'CPG'